In [1]:
from scripts.data_utils.sqlConnector import online_sql_connector , update_connector
update_connector(path = './data/DB_3rdPartySQL.h5' , connector = online_sql_connector() , trace = 1)


haitong/hf_factors
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:20:08 2023 : haitong:hf_factors:202312                    

haitong/dl_factors
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:20:53 2023 : haitong:dl_factors:202312                    

dongfang/hfq_chars
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:21:01 2023 : dongfang:hfq_chars:202312                    

dongfang/l2_chars
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:21:31 2023 : dongfang:l2_chars:202312                    

dongfang/ms_chars
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:23:29 2023 : dongfang:ms_chars:202312                    

dongfang/order_flow
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:23:33 2023 : dongfang:order_flow:202312                    

dongfang/gp
since 20231212 to 20231219, total 1 periods(Q)
Tue Dec 19 22:23:37 2023 : dongfang:gp:202312                    

dongfang/tra
since 20231212 to 20231

In [5]:
from scripts.data_utils.DataTank import DataTank , Data1D
dtank = DataTank('./data/DB_3rdPartySQL.h5' , open=True , mode = 'r')
g = dtank.get_object('haitong/hf_factors/20231213')
g

<HDF5 group "/haitong/hf_factors/20231213" (3 members)>